In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import privacy
print(tf.__version__)
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

Using TensorFlow backend.
W0711 09:33:14.304923 140304495933248 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0711 09:33:14.320287 140304495933248 deprecation_wrapper.py:119] From /home/thomas/Downloads/privacy/privacy/optimizers/dp_optimizer.py:230: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0711 09:33:14.322171 140304495933248 deprecation_wrapper.py:119] From /home/thomas/Downloads/privacy/privacy/optimizers/dp_optimizer.py:231: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0711 09:33:14.324059 140304495933248 deprecation_wrapper.py

1.14.0


In [2]:
data = pd.read_csv("datasets/hmda_va_clean.csv")
dataevensamp = pd.concat([(data[data["action_taken_name"] == 1]).sample(n=len(data[data["action_taken_name"]==0])),
                          data[data["action_taken_name"]==0]])
data = dataevensamp.sample(frac=1).iloc[:,1:]
data.head()

,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,owner_occupancy_name,loan_type_name,loan_purpose_name,lien_status_name,co_applicant_sex_name,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_abbr,state_abbr_VA,action_taken_name
83283,0.580755,0.270304,0.263382,0.390418,0.411782,0.126967,0.000000,0.260453,0,1,1,0,0,0,0,1,1,0,1,1
144080,0.672752,0.603743,0.449440,0.645359,0.656490,0.466509,1.000000,0.327474,1,1,1,1,1,0,1,1,1,0,1,1
201473,0.699785,0.639761,0.366584,0.735303,0.734265,0.414757,0.272560,0.333182,1,1,0,0,1,1,1,0,1,1,1,0
193424,0.759269,0.675982,0.406777,0.750315,0.724571,0.438675,0.362799,0.381058,1,1,1,0,1,0,0,1,1,1,1,0
94775,0.720665,0.489062,0.219632,0.569032,0.605935,0.400866,0.000000,0.317374,1,1,1,0,1,1,1,1,1,1,1,1


In [3]:
#data[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']].hist(bins=100, figsize=(15, 15))

In [4]:
# MaxMinScaling
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range = (0, 1))
#data[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']] = scaler.fit_transform(data[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']])

In [5]:
#data[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']].hist(bins=100, figsize=(15, 15))

In [12]:
predictors = data.iloc[:,0:-1].values
labels = pd.to_numeric(data.iloc[:,-1], downcast='integer').values
from keras.utils import to_categorical
labels = to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(predictors, labels, test_size=.2)
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [13]:
# 1. INSTANTIATE
#enc = sklearn.preprocessing.OneHotEncoder()

# 2. FIT
#enc.fit(data.drop(['approved'], axis=1))

# 3. Transform
#onehotlabels = enc.transform(data.drop(['approved'], axis=1)).toarray()
#predictors = pd.DataFrame(onehotlabels).values
#labels = pd.to_numeric(data.iloc[:,-1], downcast='integer').values

#x_train, x_test, y_train, y_test = train_test_split(onehotlabels, labels, test_size=.2)

In [14]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    keras.backend.get_session().run(tf.local_variables_initializer())
    return auc

In [37]:
def model_creation(optimizer='adam', loss='categorical_crossentropy'):    
    model = keras.models.Sequential([
        keras.layers.Dense(100, activation=tf.nn.relu),
        keras.layers.Dense(25, activation=tf.nn.relu),
        keras.layers.Dense(20, activation=tf.nn.relu),
        keras.layers.Dense(2, activation=tf.nn.softmax)
    ])
    model.compile(optimizer=optimizer,
             loss=loss,
             metrics=['accuracy', auc])
    return model

In [16]:
model = model_creation()
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
model.summary()

Train on 64787 samples, validate on 16197 samples
Epoch 1/5
64787/64787 [==============================] - 23s 355us/step - loss: 0.5812 - acc: 0.6975 - auc: 0.7402 - val_loss: 0.5659 - val_acc: 0.7092 - val_auc: 0.7627
Epoch 2/5
64787/64787 [==============================] - 11s 171us/step - loss: 0.5643 - acc: 0.7086 - auc: 0.7673 - val_loss: 0.5650 - val_acc: 0.7072 - val_auc: 0.7701
Epoch 3/5
64787/64787 [==============================] - 11s 166us/step - loss: 0.5601 - acc: 0.7105 - auc: 0.7719 - val_loss: 0.5601 - val_acc: 0.7136 - val_auc: 0.7736
Epoch 4/5
64787/64787 [==============================] - 11s 166us/step - loss: 0.5578 - acc: 0.7103 - auc: 0.7749 - val_loss: 0.5564 - val_acc: 0.7141 - val_auc: 0.7758
Epoch 5/5
64787/64787 [==============================] - 12s 186us/step - loss: 0.5548 - acc: 0.7119 - auc: 0.7767 - val_loss: 0.5551 - val_acc: 0.7135 - val_auc: 0.7776
_________________________________________________________________
Layer (type)                 Outpu

In [35]:
model.get_weights()

[array([[ 2.52286166e-01, -2.49711946e-01, -9.78391171e-02,
         -1.56426966e-01,  1.46167010e-01,  1.83359757e-01,
         -1.73424721e-01,  5.55609129e-02, -2.32354283e-01,
         -1.21092297e-01,  7.82404095e-03, -2.07624167e-01,
          5.39621264e-02, -1.50604080e-02, -1.32848814e-01,
          1.60886034e-01,  3.31191510e-01,  1.50362715e-01,
         -3.37562561e-02, -2.82825410e-01, -1.96428776e-01,
          2.23382607e-01, -5.96768484e-02,  1.53935710e-02,
         -2.28989050e-01, -7.42953569e-02,  2.30342522e-01,
          7.68296942e-02,  1.34737045e-02, -1.32661939e-01,
          2.36685835e-02,  2.61563778e-01,  1.38941854e-01,
          6.69113398e-02, -5.91301173e-02,  2.83082306e-01,
          1.04328021e-02,  2.25828975e-01,  9.52880159e-02,
          2.42456585e-01,  1.05253728e-02,  7.37253502e-02,
          8.52136388e-02, -1.81840166e-01,  6.74012525e-04,
         -1.59818649e-01, -1.16286255e-01, -8.95565748e-02,
         -3.35160568e-02,  5.25046214e-0

In [23]:
predictions = model.predict(x_test)
print(predictions)
print(np.argmax(y_test, axis=1))

[[0.5362094  0.4637906 ]
 [0.8539383  0.14606167]
 [0.7255047  0.27449533]
 ...
 [0.625373   0.37462696]
 [0.5435859  0.45641413]
 [0.8088053  0.19119473]]
[0 1 1 ... 1 0 0]


In [24]:
rounded = np.rint(predictions)

con_mat = tf.confusion_matrix(labels=np.argmax(y_test, axis=1),predictions=np.argmax(predictions, axis=1), num_classes=2, dtype=tf.int32, name=None)

with tf.Session():
   print('Confusion Matrix: \n\n', tf.Tensor.eval(con_mat,feed_dict=None, session=None))

Confusion Matrix: 

 [[1352  453]
 [ 777 1062]]


In [72]:
def kfoldcrossvalidation(alldata, split_size=5, optimizer="adam", epochs=10, batch_size=32, sample_frac=1):
    alldata = pd.DataFrame(alldata).sample(frac=sample_frac) #Shuffles Data when frac=1
    kf = KFold(n_splits=split_size)
    average_loss, average_acc, average_auc = (0,0,0)
    index = 1
    print(optimizer, "test")
    for train_index, test_index in kf.split(alldata):
        x_train = alldata.iloc[:,0:-1].values[train_index]
        y_train = to_categorical(pd.to_numeric(alldata.iloc[:,-1], downcast='integer').values[train_index])
        x_test = alldata.iloc[:,0:-1].values[test_index]
        y_test = to_categorical(pd.to_numeric(alldata.iloc[:,-1], downcast='integer').values[test_index])
        model = model_creation(optimizer=optimizer)
        model.fit(x_train, y_train, epochs=epochs, verbose=0)
        model_loss, model_acc, model_auc = model.evaluate(x_test, y_test, verbose=0)
        average_loss += model_loss/split_size
        average_acc += model_acc/split_size
        average_auc += model_auc/split_size
        print("Split", index,"Accuracy:","%.3f" % model_acc)
        index += 1
    print("Average Model Loss:", "%.3f" % average_loss)
    print("Average Model Accuracy:", "%.3f" % average_acc)
    print("Average Model AUC:", "%.3f" % average_auc)

In [ ]:
kfoldcrossvalidation(data)

In [14]:
kfoldcrossvalidation(data, split_size=10)

Split 1 Accuracy: 0.671
Split 2 Accuracy: 0.661
Split 3 Accuracy: 0.642
Split 4 Accuracy: 0.668
Split 5 Accuracy: 0.648
Split 6 Accuracy: 0.636
Split 7 Accuracy: 0.678
Split 8 Accuracy: 0.660
Split 9 Accuracy: 0.662
Split 10 Accuracy: 0.672
Average Model Loss: 0.612
Average Model Accuracy: 0.660
Average Model AUC: 0.716


In [17]:
kfoldcrossvalidation(data, split_size=10)

Split 1 Accuracy: 0.639
Split 2 Accuracy: 0.664
Split 3 Accuracy: 0.686
Split 4 Accuracy: 0.673
Split 5 Accuracy: 0.638
Split 6 Accuracy: 0.664
Split 7 Accuracy: 0.680
Split 8 Accuracy: 0.653
Split 9 Accuracy: 0.627
Split 10 Accuracy: 0.675
Average Model Loss: 0.585
Average Model Accuracy: 0.660
Average Model AUC: 0.729


In [26]:
kfoldcrossvalidation(data, split_size=10)

Split 1 Accuracy: 0.630
Split 2 Accuracy: 0.632
Split 3 Accuracy: 0.644
Split 4 Accuracy: 0.631
Split 5 Accuracy: 0.645
Split 6 Accuracy: 0.645
Split 7 Accuracy: 0.630
Split 8 Accuracy: 0.632
Split 9 Accuracy: 0.650
Split 10 Accuracy: 0.636
Average Model Loss: 0.627
Average Model Accuracy: 0.637
Average Model AUC: 0.689


In [28]:
kfoldcrossvalidation(data, split_size=10)

Split 1 Accuracy: 0.715
Split 2 Accuracy: 0.717
Split 3 Accuracy: 0.721
Split 4 Accuracy: 0.722
Split 5 Accuracy: 0.723
Split 6 Accuracy: 0.708
Split 7 Accuracy: 0.715
Split 8 Accuracy: 0.715
Split 9 Accuracy: 0.716
Split 10 Accuracy: 0.717
Average Model Loss: 0.548
Average Model Accuracy: 0.717
Average Model AUC: 0.785


In [31]:
kfoldcrossvalidation(data, split_size=10)

Split 1 Accuracy: 0.709
Split 2 Accuracy: 0.717
Split 3 Accuracy: 0.721
Split 4 Accuracy: 0.710
Split 5 Accuracy: 0.716
Split 6 Accuracy: 0.724
Split 7 Accuracy: 0.717
Split 8 Accuracy: 0.727
Split 9 Accuracy: 0.717
Split 10 Accuracy: 0.718
Average Model Loss: 0.547
Average Model Accuracy: 0.718
Average Model AUC: 0.785


In [44]:
def optimizertest(optimizers=[]):
    assert (len(optimizers) > 0), "No optimizers specified in function call"
    for o in optimizers:
        kfoldcrossvalidation(data, optimizer=o, epochs=2, sample_frac=.2)

In [39]:
#Parameters
l2_norm_clip=1.0
noise_multiplier=1.1
num_microbatches=1
batch_size=256
learning_rate=.15
epochs=5
population_size=len(data)

privateop = privacy.optimizers.dp_optimizer.DPGradientDescentGaussianOptimizer(l2_norm_clip=l2_norm_clip,
                                                                              noise_multiplier=noise_multiplier,
                                                                              num_microbatches=num_microbatches,
                                                                              learning_rate=learning_rate,
                                                                              unroll_microbatches=True)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, reduction=tf.losses.Reduction.NONE)

In [73]:
optimizertest(optimizers=["adam", privateop])

adam test
Split 1 Accuracy: 0.706
Split 2 Accuracy: 0.696
Split 3 Accuracy: 0.701
Split 4 Accuracy: 0.695
Split 5 Accuracy: 0.692
Average Model Loss: 0.583
Average Model Accuracy: 0.698
Average Model AUC: 0.743
<privacy.optimizers.dp_optimizer.make_gaussian_optimizer_class.<locals>.DPGaussianOptimizerClass object at 0x7f9516b34dd8> test
Split 1 Accuracy: 0.510
Split 2 Accuracy: 0.498
Split 3 Accuracy: 0.496
Split 4 Accuracy: 0.495
Split 5 Accuracy: 0.502
Average Model Loss: 0.742
Average Model Accuracy: 0.500
Average Model AUC: 0.513


In [40]:
model = model_creation(optimizer=privateop, loss=loss)
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), batch_size=256)
model.summary()

Train on 64787 samples, validate on 16197 samples
Epoch 1/50
64787/64787 [==============================] - 3s 39us/step - loss: 343.2967 - acc: 0.5038 - auc: 0.4944 - val_loss: 830.5654 - val_acc: 0.5506 - val_auc: 0.5096
Epoch 2/50
64787/64787 [==============================] - 2s 28us/step - loss: 1776.0211 - acc: 0.5536 - auc: 0.5235 - val_loss: 2277.0548 - val_acc: 0.6323 - val_auc: 0.5370
Epoch 3/50
64787/64787 [==============================] - 2s 30us/step - loss: 2186.3911 - acc: 0.5805 - auc: 0.5486 - val_loss: 1594.3379 - val_acc: 0.4442 - val_auc: 0.5495
Epoch 4/50
64787/64787 [==============================] - 2s 30us/step - loss: 2003.9375 - acc: 0.5566 - auc: 0.5470 - val_loss: 3506.2222 - val_acc: 0.5186 - val_auc: 0.5475
Epoch 5/50
64787/64787 [==============================] - 2s 29us/step - loss: 4374.1005 - acc: 0.5398 - auc: 0.5458 - val_loss: 8798.5959 - val_acc: 0.4977 - val_auc: 0.5445
Epoch 6/50
64787/64787 [==============================] - 2s 30us/step - loss

64787/64787 [==============================] - 1s 21us/step - loss: 433.6558 - acc: 0.4993 - auc: 0.5129 - val_loss: 871.5193 - val_acc: 0.4982 - val_auc: 0.5128
Epoch 49/50
64787/64787 [==============================] - 1s 19us/step - loss: 606.6972 - acc: 0.5008 - auc: 0.5126 - val_loss: 239.2328 - val_acc: 0.5018 - val_auc: 0.5125
Epoch 50/50
64787/64787 [==============================] - 1s 19us/step - loss: 579.3308 - acc: 0.5007 - auc: 0.5124 - val_loss: 369.9285 - val_acc: 0.4982 - val_auc: 0.5123
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 100)               2000      
_________________________________________________________________
dense_42 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_43 (Dense)             (None, 20)                520       
____________________________

In [30]:
predictions = model.predict(x_test)
rounded = np.rint(predictions)

con_mat = tf.confusion_matrix(labels=np.argmax(y_test, axis=1),predictions=np.argmax(predictions, axis=1), num_classes=2, dtype=tf.int32, name=None)

with tf.Session():
   print('Confusion Matrix: \n\n', tf.Tensor.eval(con_mat,feed_dict=None, session=None))

Confusion Matrix: 

 [[5059 3013]
 [4346 3779]]
